In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import plotly.express as px


In [72]:
!pip install pandas_datareader

In [6]:
pip install pandas_ta

     |████████████████████████████████| 115 kB 1.6 MB/s eta 0:00:01
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218924 sha256=01c2058fd58f0319d57ac6e193c27fe53f6a9f6f6a45269597a8d5c7aba9a9e6
  Stored in directory: /home/antash/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta
Note: you may need to restart the kernel to use updated packages.


In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import statsmodels.api as sm
import datetime as dt
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [3]:
stock = yf.Ticker('BPCL')

In [4]:
stock

yfinance.Ticker object <BPCL>

In [76]:
df = stock.history(period="5y")

BPCL: No price data found, symbol may be delisted (period=5y)


In [35]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-14 00:00:00+05:30,7311.000000,7431.700195,7251.850098,7402.549805,46998,0.0,0.0
2019-01-15 00:00:00+05:30,7438.700195,7465.000000,7321.000000,7349.549805,36810,0.0,0.0
2019-01-16 00:00:00+05:30,7365.000000,7384.750000,7291.049805,7305.250000,38526,0.0,0.0
2019-01-17 00:00:00+05:30,7325.000000,7397.950195,7310.250000,7333.700195,26924,0.0,0.0
2019-01-18 00:00:00+05:30,7365.000000,7375.000000,7324.700195,7353.350098,22213,0.0,0.0
...,...,...,...,...,...,...,...
2024-01-08 00:00:00+05:30,10018.650391,10125.000000,9914.000000,9947.299805,15007,0.0,0.0
2024-01-09 00:00:00+05:30,10019.650391,10040.000000,9891.799805,10023.049805,25518,0.0,0.0
2024-01-10 00:00:00+05:30,10039.900391,10055.049805,9949.700195,10002.049805,20910,0.0,0.0


In [36]:
stock.info

{'address1': '1, Nelson Mandela Road',
 'address2': 'Vasant Kunj',
 'city': 'New Delhi',
 'zip': '110070',
 'country': 'India',
 'phone': '91 11 4678 1000',
 'fax': '91 11 4615 0275',
 'website': 'https://www.marutisuzuki.com',
 'industry': 'Auto Manufacturers',
 'industryKey': 'auto-manufacturers',
 'industryDisp': 'Auto Manufacturers',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Maruti Suzuki India Limited manufactures, purchases, and sells motor vehicles, components, and spare parts primarily in India. The company offers passenger vehicles, utility vehicles, and multi-purpose vehicles. It is also involved in the facilitation of pre-owned car sales, fleet management, and car financing activities. In addition, the company offers driving school, accessories, insurance, and financing products and services. It also exports its products to Chile, Ivory Coast, Saudi Arabia, Ethiopia, South Africa, and inter

In [3]:
nifty500 = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_500', header=0)[2]

In [4]:
nifty500

,Sl.No,Company Name,Industry,Symbol,Series,ISIN Code
0,1,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,2,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,3,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,4,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,5,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026
...,...,...,...,...,...,...
496,497,Zensar Technologies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027
497,498,Zomato Ltd.,Consumer Services,ZOMATO,EQ,INE758T01015
498,499,Zydus Lifesciences Ltd.,Healthcare,ZYDUSLIFE,EQ,INE010B01027
499,500,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010


In [5]:
symbols_list = nifty500['Symbol'].unique().tolist()
symbols_list = [item + ".NS" for item in symbols_list]

In [6]:
end_date = '2024-01-12'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [7]:
df = yf.download(tickers= symbols_list,
                start = start_date,
                end = end_date)

[*********************100%%**********************]  501 of 501 completed


2 Failed downloads:
['AMARAJABAT.NS', 'MFL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [8]:
df

Adj Close                                                       \
             360ONE.NS    3MINDIA.NS AARTIDRUGS.NS AARTIIND.NS     AAVAS.NS   
Date                                                                          
2016-01-14         NaN  10004.662109    133.225739  117.769081          NaN   
2016-01-15         NaN   9817.962891    131.582352  116.890457          NaN   
2016-01-18         NaN   9460.190430    121.819283  104.039307          NaN   
2016-01-19         NaN   9580.152344    121.186256  107.624031          NaN   
2016-01-20         NaN   9461.391602    121.867966  111.396217          NaN   
...                ...           ...           ...         ...          ...   
2024-01-05  662.450012  34696.949219    509.700012  629.299988  1566.050049   
2024-01-08  672.450012  34070.601562    502.000000  598.750000  1552.449951   
2024-01-09  674.750000  34155.000000    510.000000  603.250000  1552.750000   
2024-01-10  683.150024  34046.648438    518.650024  603.599976  1568.699951   
2024-01-11  680.349976  34098.648438    518.200012  620.000000  1578.750000   

                                                                             \
                 ABB.NS ABBOTINDIA.NS ABCAPITAL.NS    ABFRL.NS       ACC.NS   
Date                                                                          
2016-01-14   972.976135   5028.819824          NaN  215.699997  1132.163940   
2016-01-15   973.024170   5078.498047          NaN  212.600006  1096.969238   
2016-01-18   964.240906   4895.499512          NaN  200.300003  1092.196411   
2016-01-19   978.591797   4942.551270          NaN  211.149994  1102.054443   
2016-01-20   953.777527   4793.516113          NaN  218.800003  1081.490479   
...                 ...           ...          ...         ...          ...   
2024-01-05  4832.350098  23884.599609   175.100006  238.100006  2377.449951   
2024-01-08  4752.950195  23658.099609   175.399994  230.800003  2354.750000   
2024-01-09  4920.399902  24231.900391   175.149994  229.449997  2309.649902   
2024-01-10  4935.700195  24609.250000   173.300003  226.750000  2306.600098   
2024-01-11  4871.850098  25339.650391   180.550003  233.949997  2321.750000   

            ...      Volume                                              \
            ... WESTLIFE.NS WHIRLPOOL.NS WIPRO.NS YESBANK.NS    ZEEL.NS   
Date        ...                                                           
2016-01-14  ...         NaN        58302  2919332   17869410    2107954   
2016-01-15  ...         NaN        14863  1667087   18152225    5656028   
2016-01-18  ...         NaN        39763  4515961   18905815    2733395   
2016-01-19  ...         NaN       150586  2553140   24393180    1175138   
2016-01-20  ...         NaN        62138  3119055   26445660    2017744   
...         ...         ...          ...      ...        ...        ...   
2024-01-05  ...     51782.0       171282  8444565  561257556   20893253   
2024-01-08  ...     96957.0       104746  5041443  546082056   11033896   
2024-01-09  ...     84103.0        84641  7232524  474325248  157297159   
2024-01-10  ...    107692.0        30820  4437338  706576963   59317188   
2024-01-11  ...     93253.0        43556  4319409  410909551   19043312   

                                                                             
           ZENSARTECH.NS ZFCVINDIA.NS   ZOMATO.NS ZYDUSLIFE.NS ZYDUSWELL.NS  
Date                                                                         
2016-01-14        194360       9306.0         NaN      2398811         2628  
2016-01-15        146685       5979.0         NaN      2081169         4477  
2016-01-18        226155       5687.0         NaN      2290276         8746  
2016-01-19        307765       3342.0         NaN      1729854         6741  
2016-01-20       1207150       3311.0         NaN      2113279         4426  
...                  ...          ...         ...          ...          ...  
2024-01-05       1184596       4336.0  5560892

In [9]:
df.stack()

Adj Close         Close          High  \
Date                                                                 
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   Low          Open      Volume  
Date                                                              
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0  
           AARTIDRUGS.NS    133.750000    135.912506     28804.0  
           AARTIIND.NS      118.581184    121.983109    119122.0  
           ABB.NS          1001.000000   1015.000000     26224.0  
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0  
...                                ...           ...         ...  
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0  
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0  
           ZOMATO.NS        135.050003    135.300003  41809415.0  
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0  
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0  

[849817 rows x 6 columns]

In [10]:
df = df.stack()

In [11]:
df.index.names = ['date', 'ticker']

In [12]:
df.columns = df.columns.str.lower()

In [13]:
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  
date       ticker                                                 
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0  
           AARTIDRUGS.NS    133.750000    135.912506     28804.0  
           AARTIIND.NS      118.581184    121.983109    119122.0  
           ABB.NS          1001.000000   1015.000000     26224.0  
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0  
...                                ...           ...         ...  
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0  
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0  
           ZOMATO.NS        135.050003    135.300003  41809415.0  
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0  
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0  

[849817 rows x 6 columns]

### Calculate feature and technical indicator

In [14]:
df['graman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open']))**2)

In [15]:
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol  
date       ticker                           
2016-01-14 3MINDIA.NS            -0.000532  
           AARTIDRUGS.NS          0.000350  
           AARTIIND.NS            0.000244  
           ABB.NS                -0.000283  
           ABBOTINDIA.NS         -0.002291  
...                                    ...  
2024-01-11 ZENSARTECH.NS          0.000233  
           ZFCVINDIA.NS           0.000102  
           ZOMATO.NS              0.000224  
           ZYDUSLIFE.NS           0.000110  
           ZYDUSWELL.NS           0.000088  

[849817 rows x 7 columns]

In [16]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [17]:
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi  
date       ticker                                      
2016-01-14 3MINDIA.NS            -0.000532        NaN  
           AARTIDRUGS.NS          0.000350        NaN  
           AARTIIND.NS            0.000244        NaN  
           ABB.NS                -0.000283        NaN  
           ABBOTINDIA.NS         -0.002291        NaN  
...                                    ...        ...  
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  
           ZFCVINDIA.NS           0.000102  50.480529  
           ZOMATO.NS              0.000224  67.001645  
           ZYDUSLIFE.NS           0.000110  63.909881  
           ZYDUSWELL.NS           0.000088  56.341993  

[849817 rows x 8 columns]

### Bollingers Band

In [18]:
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])
df


adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2016-01-14 3MINDIA.NS            -0.000532        NaN       NaN       NaN   
           AARTIDRUGS.NS          0.000350        NaN       NaN       NaN   
           AARTIIND.NS            0.000244        NaN       NaN       NaN   
           ABB.NS                -0.000283        NaN       NaN       NaN   
           ABBOTINDIA.NS         -0.002291        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  6.299932  6.384950   
           ZFCVINDIA.NS           0.000102  50.480529  9.629849  9.669597   
           ZOMATO.NS              0.000224  67.001645  4.798648  4.863176   
           ZYDUSLIFE.NS           0.000110  63.909881  6.475204  6.537072   
           ZYDUSWELL.NS           0.000088  56.341993  7.348625  7.399999   

                           bb_high  
date       ticker                   
2016-01-14 3MINDIA.NS          NaN  
           AARTIDRUGS.NS       NaN  
           AARTIIND.NS         NaN  
           ABB.NS              NaN  
           ABBOTINDIA.NS       NaN  
...                            ...  
2024-01-11 ZENSARTECH.NS  6.469969  
           ZFCVINDIA.NS   9.709345  
           ZOMATO.NS      4.927705  
           ZYDUSLIFE.NS   6.598939  
           ZYDUSWELL.NS   7.451374  

[849817 rows x 11 columns]

### ATR Calculation

In [19]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                       low= stock_data['low'],
                       close = stock_data['close'],
                       length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2016-01-14 3MINDIA.NS            -0.000532        NaN       NaN       NaN   
           AARTIDRUGS.NS          0.000350        NaN       NaN       NaN   
           AARTIIND.NS            0.000244        NaN       NaN       NaN   
           ABB.NS                -0.000283        NaN       NaN       NaN   
           ABBOTINDIA.NS         -0.002291        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  6.299932  6.384950   
           ZFCVINDIA.NS           0.000102  50.480529  9.629849  9.669597   
           ZOMATO.NS              0.000224  67.001645  4.798648  4.863176   
           ZYDUSLIFE.NS           0.000110  63.909881  6.475204  6.537072   
           ZYDUSWELL.NS           0.000088  56.341993  7.348625  7.399999   

                           bb_high       atr  
date       ticker                             
2016-01-14 3MINDIA.NS          NaN       NaN  
           AARTIDRUGS.NS       NaN       NaN  
           AARTIIND.NS         NaN       NaN  
           ABB.NS              NaN       NaN  
           ABBOTINDIA.NS       NaN       NaN  
...                            ...       ...  
2024-01-11 ZENSARTECH.NS  6.469969  2.474345  
           ZFCVINDIA.NS   9.709345  2.228942  
           ZOMATO.NS      4.927705  0.108510  
           ZYDUSLIFE.NS   6.598939  0.698430  
           ZYDUSWELL.NS   7.451374 -0.490535  

[849817 rows x 12 columns]

### MaCD Calculation

In [20]:
def compute_macd(close):
    macd = pandas_ta.macd(close = close, length= 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2016-01-14 3MINDIA.NS            -0.000532        NaN       NaN       NaN   
           AARTIDRUGS.NS          0.000350        NaN       NaN       NaN   
           AARTIIND.NS            0.000244        NaN       NaN       NaN   
           ABB.NS                -0.000283        NaN       NaN       NaN   
           ABBOTINDIA.NS         -0.002291        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  6.299932  6.384950   
           ZFCVINDIA.NS           0.000102  50.480529  9.629849  9.669597   
           ZOMATO.NS              0.000224  67.001645  4.798648  4.863176   
           ZYDUSLIFE.NS           0.000110  63.909881  6.475204  6.537072   
           ZYDUSWELL.NS           0.000088  56.341993  7.348625  7.399999   

                           bb_high       atr      macd  
date       ticker                                       
2016-01-14 3MINDIA.NS          NaN       NaN       NaN  
           AARTIDRUGS.NS       NaN       NaN       NaN  
           AARTIIND.NS         NaN       NaN       NaN  
           ABB.NS              NaN       NaN       NaN  
           ABBOTINDIA.NS       NaN       NaN       NaN  
...                            ...       ...       ...  
2024-01-11 ZENSARTECH.NS  6.469969  2.474345  0.429331  
           ZFCVINDIA.NS   9.709345  2.228942  0.248212  
           ZOMATO.NS      4.927705  0.108510  1.270159  
           ZYDUSLIFE.NS   6.598939  0.698430  1.925560  
           ZYDUSWELL.NS   7.451374 -0.490535  0.823006  

[849817 rows x 13 columns]

### Dollar Volume

In [21]:
df['dollar_volume'] =  (df['adj close']* df['volume'])/1e6

In [22]:
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2016-01-14 3MINDIA.NS            -0.000532        NaN       NaN       NaN   
           AARTIDRUGS.NS          0.000350        NaN       NaN       NaN   
           AARTIIND.NS            0.000244        NaN       NaN       NaN   
           ABB.NS                -0.000283        NaN       NaN       NaN   
           ABBOTINDIA.NS         -0.002291        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  6.299932  6.384950   
           ZFCVINDIA.NS           0.000102  50.480529  9.629849  9.669597   
           ZOMATO.NS              0.000224  67.001645  4.798648  4.863176   
           ZYDUSLIFE.NS           0.000110  63.909881  6.475204  6.537072   
           ZYDUSWELL.NS           0.000088  56.341993  7.348625  7.399999   

                           bb_high       atr      macd  dollar_volume  
date       ticker                                                      
2016-01-14 3MINDIA.NS          NaN       NaN       NaN       4.362033  
           AARTIDRUGS.NS       NaN       NaN       NaN       3.837434  
           AARTIIND.NS         NaN       NaN       NaN      14.028888  
           ABB.NS              NaN       NaN       NaN      25.515326  
           ABBOTINDIA.NS       NaN       NaN       NaN      12.481531  
...                            ...       ...       ...            ...  
2024-01-11 ZENSARTECH.NS  6.469969  2.474345  0.429331     557.292701  
           ZFCVINDIA.NS   9.709345  2.228942  0.248212      26.828107  
           ZOMATO.NS      4.927705  0.108510  1.270159    5784.332820  
           ZYDUSLIFE.NS   6.598939  0.698430  1.925560     377.081507  
           ZYDUSWELL.NS   7.451374 -0.490535  0.823006      39.878036  

[849817 rows x 14 columns]

In [23]:
df.unstack('ticker')['adj close'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

dollar_volume
date       ticker                      
2016-01-31 3MINDIA.NS       9508.310458
           AARTIDRUGS.NS     126.060045
           AARTIIND.NS       112.661412
           ABB.NS            948.742515
           ABBOTINDIA.NS    4821.572754
...                                 ...
2024-01-31 KFINTECH.NS       503.538893
           NSLNISP.NS         50.205556
           MANKIND.NS       2087.661106
           KPIL.NS           730.555556
           ADANIENSOL.NS    1142.455553

[41813 rows x 1 columns]

In [24]:
df = df.drop(columns=['dollar volume'])

KeyError: "['dollar volume'] not found in axis"

In [25]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'high', 'low', 'open', 'close']]

In [26]:
last_cols

['adj close',
 'graman_klass_vol',
 'rsi',
 'bb_low',
 'bb_mid',
 'bb_high',
 'atr',
 'macd']

In [27]:
df.unstack()[last_cols].resample('M').last().stack('ticker')

adj close       atr   bb_high    bb_low  \
date       ticker                                                      
2016-01-31 3MINDIA.NS      9501.972656       NaN       NaN       NaN   
           AARTIDRUGS.NS    128.185959       NaN       NaN       NaN   
           AARTIIND.NS      115.437813       NaN       NaN       NaN   
           ABB.NS           932.179504       NaN       NaN       NaN   
           ABBOTINDIA.NS   4752.328125       NaN       NaN       NaN   
...                                ...       ...       ...       ...   
2024-01-31 ZENSARTECH.NS    563.150024  2.474345  6.469969  6.299932   
           ZFCVINDIA.NS   15902.849609  2.228942  9.709345  9.629849   
           ZOMATO.NS        138.350006  0.108510  4.927705  4.798648   
           ZYDUSLIFE.NS     704.250000  0.698430  6.598939  6.475204   
           ZYDUSWELL.NS    1643.099976 -0.490535  7.451374  7.348625   

                            bb_mid  graman_klass_vol      macd        rsi  
date       ticker                                                          
2016-01-31 3MINDIA.NS          NaN          0.000465       NaN        NaN  
           AARTIDRUGS.NS       NaN         -0.000152       NaN        NaN  
           AARTIIND.NS         NaN         -0.000350       NaN        NaN  
           ABB.NS              NaN         -0.000393       NaN        NaN  
           ABBOTINDIA.NS       NaN         -0.000333       NaN        NaN  
...                            ...               ...       ...        ...  
2024-01-31 ZENSARTECH.NS  6.384950          0.000233  0.429331  49.579103  
           ZFCVINDIA.NS   9.669597          0.000102  0.248212  50.480529  
           ZOMATO.NS      4.863176          0.000224  1.270159  67.001645  
           ZYDUSLIFE.NS   6.537072          0.000110  1.925560  63.909881  
           ZYDUSWELL.NS   7.399999          0.000088  0.823006  56.341993  

[41813 rows x 8 columns]

In [28]:
data = pd.concat([df.unstack('ticker')['adj close'].resample('M').mean().stack('ticker').to_frame('dollar_volume'), df.unstack()[last_cols].resample('M').last().stack('ticker')], axis=1).dropna()

In [29]:
data

dollar_volume     adj close       atr   bb_high  \
date       ticker                                                           
2016-02-29 3MINDIA.NS      10218.828125   9983.747070 -0.743123  9.331180   
           AARTIDRUGS.NS     118.376846    112.808098 -0.691214  4.850360   
           AARTIIND.NS       106.690429    100.248024 -1.192568  4.773495   
           ABB.NS           1032.020418    972.688049  0.069312  7.019730   
           ABBOTINDIA.NS    4562.260091   4490.473633 -0.753433  8.464671   
...                                 ...           ...       ...       ...   
2024-01-31 ZENSARTECH.NS     578.727783    563.150024  2.474345  6.469969   
           ZFCVINDIA.NS    16114.144314  15902.849609  2.228942  9.709345   
           ZOMATO.NS         131.550001    138.350006  0.108510  4.927705   
           ZYDUSLIFE.NS      708.894457    704.250000  0.698430  6.598939   
           ZYDUSWELL.NS     1670.572211   1643.099976 -0.490535  7.451374   

                            bb_low    bb_mid  graman_klass_vol      macd  \
date       ticker                                                          
2016-02-29 3MINDIA.NS     9.137679  9.234429         -0.000323  0.208045   
           AARTIDRUGS.NS  4.704185  4.777273          0.000489 -0.385263   
           AARTIIND.NS    4.576438  4.674966         -0.000051 -0.442542   
           ABB.NS         6.866371  6.943050          0.002594  0.037214   
           ABBOTINDIA.NS  8.381688  8.423180         -0.001563 -0.741860   
...                            ...       ...               ...       ...   
2024-01-31 ZENSARTECH.NS  6.299932  6.384950          0.000233  0.429331   
           ZFCVINDIA.NS   9.629849  9.669597          0.000102  0.248212   
           ZOMATO.NS      4.798648  4.863176          0.000224  1.270159   
           ZYDUSLIFE.NS   6.475204  6.537072          0.000110  1.925560   
           ZYDUSWELL.NS   7.348625  7.399999          0.000088  0.823006   

                                rsi  
date       ticker                    
2016-02-29 3MINDIA.NS     48.114397  
           AARTIDRUGS.NS  36.415393  
           AARTIIND.NS    34.827972  
           ABB.NS         44.864961  
           ABBOTINDIA.NS  38.264953  
...                             ...  
2024-01-31 ZENSARTECH.NS  49.579103  
           ZFCVINDIA.NS   50.480529  
           ZOMATO.NS      67.001645  
           ZYDUSLIFE.NS   63.909881  
           ZYDUSWELL.NS   56.341993  

[41274 rows x 9 columns]

In [30]:
data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

In [31]:
data['dollar_volume_rank'] = data.groupby('date')['dollar_volume'].rank(ascending=False)

In [32]:
data = data[data['dollar_volume_rank'] <= 150].drop(columns=['dollar_volume_rank', 'dollar_volume'],axis=1)

In [33]:
data

adj close       atr   bb_high    bb_low  \
date       ticker                                                      
2021-01-31 3MINDIA.NS     18426.199219 -0.086960  9.983237  9.836033   
           ABB.NS          1372.382446 -0.404142  7.252138  7.101812   
           ABBOTINDIA.NS  13832.707031 -0.004425  9.653404  9.521063   
           ACC.NS          1533.170288  0.250481  7.462298  7.322902   
           AIAENG.NS       1930.247559  0.820972  7.674005  7.537406   
...                                ...       ...       ...       ...   
2024-01-31 VMART.NS        2113.850098 -0.467116  7.728098  7.500005   
           VOLTAS.NS       1055.699951  0.492112  6.982007  6.753808   
           WHIRLPOOL.NS    1355.199951 -1.098621  7.224920  7.202271   
           ZFCVINDIA.NS   15902.849609  2.228942  9.709345  9.629849   
           ZYDUSWELL.NS    1643.099976 -0.490535  7.451374  7.348625   

                            bb_mid  graman_klass_vol      macd        rsi  
date       ticker                                                          
2021-01-31 3MINDIA.NS     9.909635         -0.000783 -1.353472  32.610305  
           ABB.NS         7.176975          0.000422  0.833824  66.227095  
           ABBOTINDIA.NS  9.587233         -0.000418 -1.776383  30.862246  
           ACC.NS         7.392600         -0.001195 -0.283431  42.243454  
           AIAENG.NS      7.605705          0.000243 -0.214055  46.971165  
...                            ...               ...       ...        ...  
2024-01-31 VMART.NS       7.614051          0.000297  1.261121  62.696453  
           VOLTAS.NS      6.867907          0.000479  2.535716  77.325351  
           WHIRLPOOL.NS   7.213596          0.000065 -0.695997  37.808308  
           ZFCVINDIA.NS   9.669597          0.000102  0.248212  50.480529  
           ZYDUSWELL.NS   7.399999          0.000088  0.823006  56.341993  

[5550 rows x 8 columns]

In [34]:
lags = [1,2,3,6,9,12]

In [35]:
outlier_cutoff = 0.005

In [36]:
df

adj close         close          high  \
date       ticker                                                    
2016-01-14 3MINDIA.NS     10004.662109  10403.900391  10600.000000   
           AARTIDRUGS.NS    133.225739    136.800003    138.062500   
           AARTIIND.NS      117.769081    122.141052    123.173775   
           ABB.NS           972.976135   1013.599976   1030.000000   
           ABBOTINDIA.NS   5028.819824   5360.000000   5450.049805   
...                                ...           ...           ...   
2024-01-11 ZENSARTECH.NS    563.150024    563.150024    574.000000   
           ZFCVINDIA.NS   15902.849609  15902.849609  16071.049805   
           ZOMATO.NS        138.350006    138.350006    139.000000   
           ZYDUSLIFE.NS     704.250000    704.250000    711.599976   
           ZYDUSWELL.NS    1643.099976   1643.099976   1654.900024   

                                   low          open      volume  \
date       ticker                                                  
2016-01-14 3MINDIA.NS     10390.000000  10450.000000       436.0   
           AARTIDRUGS.NS    133.750000    135.912506     28804.0   
           AARTIIND.NS      118.581184    121.983109    119122.0   
           ABB.NS          1001.000000   1015.000000     26224.0   
           ABBOTINDIA.NS   5340.100098   5450.000000      2482.0   
...                                ...           ...         ...   
2024-01-11 ZENSARTECH.NS    561.000000    568.049988    989599.0   
           ZFCVINDIA.NS   15800.000000  16071.049805      1687.0   
           ZOMATO.NS        135.050003    135.300003  41809415.0   
           ZYDUSLIFE.NS     700.900024    706.599976    535437.0   
           ZYDUSWELL.NS    1633.000000   1644.800049     24270.0   

                          graman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2016-01-14 3MINDIA.NS            -0.000532        NaN       NaN       NaN   
           AARTIDRUGS.NS          0.000350        NaN       NaN       NaN   
           AARTIIND.NS            0.000244        NaN       NaN       NaN   
           ABB.NS                -0.000283        NaN       NaN       NaN   
           ABBOTINDIA.NS         -0.002291        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-01-11 ZENSARTECH.NS          0.000233  49.579103  6.299932  6.384950   
           ZFCVINDIA.NS           0.000102  50.480529  9.629849  9.669597   
           ZOMATO.NS              0.000224  67.001645  4.798648  4.863176   
           ZYDUSLIFE.NS           0.000110  63.909881  6.475204  6.537072   
           ZYDUSWELL.NS           0.000088  56.341993  7.348625  7.399999   

                           bb_high       atr      macd  dollar_volume  
date       ticker                                                      
2016-01-14 3MINDIA.NS          NaN       NaN       NaN       4.362033  
           AARTIDRUGS.NS       NaN       NaN       NaN       3.837434  
           AARTIIND.NS         NaN       NaN       NaN      14.028888  
           ABB.NS              NaN       NaN       NaN      25.515326  
           ABBOTINDIA.NS       NaN       NaN       NaN      12.481531  
...                            ...       ...       ...            ...  
2024-01-11 ZENSARTECH.NS  6.469969  2.474345  0.429331     557.292701  
           ZFCVINDIA.NS   9.709345  2.228942  0.248212      26.828107  
           ZOMATO.NS      4.927705  0.108510  1.270159    5784.332820  
           ZYDUSLIFE.NS   6.598939  0.698430  1.925560     377.081507  
           ZYDUSWELL.NS   7.451374 -0.490535  0.823006      39.878036  

[849817 rows x 14 columns]

In [37]:
px = df.xs('ZOMATO.NS', level=1)

In [38]:
px

,adj close,close,high,low,open,volume,graman_klass_vol,rsi,bb_low,bb_mid,bb_high,atr,macd,dollar_volume
date,,,,,,,,,,,,,,
2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0,0.015185,NaN,NaN,NaN,NaN,NaN,NaN,87556.806540
2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0,0.004994,NaN,NaN,NaN,NaN,NaN,NaN,35123.658541
2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0,0.009039,NaN,NaN,NaN,NaN,NaN,NaN,31941.437043
2021-07-28,131.199997,131.199997,135.000000,123.550003,131.000000,159793731.0,0.003927,NaN,NaN,NaN,NaN,NaN,NaN,20964.937020
2021-07-29,141.550003,141.550003,144.000000,132.199997,134.949997,117973089.0,0.002774,NaN,NaN,NaN,NaN,NaN,NaN,16699.091108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-05,133.300003,133.300003,134.350006,128.949997,130.899994,55608923.0,0.000714,63.051244,4.771896,4.837744,4.903592,0.234642,0.948815,7412.669606
2024-01-08,132.850006,132.850006,133.649994,130.000000,133.649994,38912423.0,0.000369,62.418695,4.775210,4.842811,4.910412,0.201178,1.016357,5169.515633
2024-01-09,134.300003,134.300003,135.000000,132.600006,133.199997,33803263.0,0.000135,63.655422,4.781709,4.849041,4.916372,0.119047,1.093033,4539.778324


In [39]:
for lag in lags:
    px[f'return_{lag}m'] = (px['adj close']
                            .pct_change(lag)
                            .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1 - outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1) 
                           )

In [40]:
px

,adj close,close,high,low,open,volume,graman_klass_vol,rsi,bb_low,bb_mid,bb_high,atr,macd,dollar_volume,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m
date,,,,,,,,,,,,,,,,,,,,
2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0,0.015185,NaN,NaN,NaN,NaN,NaN,NaN,87556.806540,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0,0.004994,NaN,NaN,NaN,NaN,NaN,NaN,35123.658541,0.116270,NaN,NaN,NaN,NaN,NaN
2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0,0.009039,NaN,NaN,NaN,NaN,NaN,NaN,31941.437043,-0.055101,0.027016,NaN,NaN,NaN,NaN
2021-07-28,131.199997,131.199997,135.000000,123.550003,131.000000,159793731.0,0.003927,NaN,NaN,NaN,NaN,NaN,NaN,20964.937020,-0.012792,-0.034178,0.013572,NaN,NaN,NaN
2021-07-29,141.550003,141.550003,144.000000,132.199997,134.949997,117973089.0,0.002774,NaN,NaN,NaN,NaN,NaN,NaN,16699.091108,0.078887,0.032030,0.002128,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-05,133.300003,133.300003,134.350006,128.949997,130.899994,55608923.0,0.000714,63.051244,4.771896,4.837744,4.903592,0.234642,0.948815,7412.669606,0.027360,0.022292,0.011775,0.013219,0.004083,0.001931
2024-01-08,132.850006,132.850006,133.649994,130.000000,133.649994,38912423.0,0.000369,62.418695,4.775210,4.842811,4.910412,0.201178,1.016357,5169.515633,-0.003376,0.011876,0.013663,0.011965,0.006790,0.005256
2024-01-09,134.300003,134.300003,135.000000,132.600006,133.199997,33803263.0,0.000135,63.655422,4.781709,4.849041,4.916372,0.119047,1.093033,4539.778324,0.010915,0.003744,0.011555,0.012708,0.006185,0.004438


In [43]:
def compute_return(df):
    outlier_cutoff = 0.005
    lags = [1,2,3,6,9,12]
    
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                      upper=x.quantile(1 - outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1) 
                               )
    return df

data = data.groupby(level=1, group_keys=False).apply(compute_return).dropna()
data

adj close       atr    bb_high     bb_low  \
date       ticker                                                        
2022-01-31 3MINDIA.NS     24030.943359  0.352469  10.127953  10.056047   
           ABB.NS          2282.991211  1.692573   7.790041   7.678768   
           ABBOTINDIA.NS  15779.435547  1.061336   9.865300   9.639898   
           ACC.NS          2201.573486  1.143787   7.736432   7.634318   
           AIAENG.NS       1874.600464 -0.384768   7.554291   7.497952   
...                                ...       ...        ...        ...   
2024-01-31 VMART.NS        2113.850098 -0.467116   7.728098   7.500005   
           VOLTAS.NS       1055.699951  0.492112   6.982007   6.753808   
           WHIRLPOOL.NS    1355.199951 -1.098621   7.224920   7.202271   
           ZFCVINDIA.NS   15902.849609  2.228942   9.709345   9.629849   
           ZYDUSWELL.NS    1643.099976 -0.490535   7.451374   7.348625   

                             bb_mid  graman_klass_vol      macd        rsi  \
date       ticker                                                            
2022-01-31 3MINDIA.NS     10.092000          0.000382 -0.613854  49.179068   
           ABB.NS          7.734405          0.001867  0.343413  53.127000   
           ABBOTINDIA.NS   9.752599         -0.000149 -3.395687  28.642637   
           ACC.NS          7.685375          0.000357 -0.367082  52.549831   
           AIAENG.NS       7.526122          0.000221  0.044066  53.935684   
...                             ...               ...       ...        ...   
2024-01-31 VMART.NS        7.614051          0.000297  1.261121  62.696453   
           VOLTAS.NS       6.867907          0.000479  2.535716  77.325351   
           WHIRLPOOL.NS    7.213596          0.000065 -0.695997  37.808308   
           ZFCVINDIA.NS    9.669597          0.000102  0.248212  50.480529   
           ZYDUSWELL.NS    7.399999          0.000088  0.823006  56.341993   

                          return_1m  return_2m  return_3m  return_6m  \
date       ticker                                                      
2022-01-31 3MINDIA.NS     -0.014625  -0.000889  -0.005420   0.002211   
           ABB.NS          0.030198   0.056415   0.027709   0.051838   
           ABBOTINDIA.NS  -0.153258  -0.076751  -0.064170  -0.027362   
           ACC.NS          0.031592  -0.000142  -0.006854  -0.007689   
           AIAENG.NS       0.008839  -0.002282  -0.005005  -0.008760   
...                             ...        ...        ...        ...   
2024-01-31 VMART.NS        0.054447   0.101177   0.088110  -0.012506   
           VOLTAS.NS       0.079062   0.124251   0.077659   0.050438   
           WHIRLPOOL.NS   -0.008523  -0.071292  -0.062022  -0.011732   
           ZFCVINDIA.NS   -0.006500   0.000055   0.004189   0.034767   
           ZYDUSWELL.NS   -0.022604   0.029097   0.023778   0.018814   

                          return_9m  return_12m  
date       ticker                                
2022-01-31 3MINDIA.NS     -0.002408    0.022377  
           ABB.NS          0.057625    0.043324  
           ABBOTINDIA.NS   0.007623    0.011033  
           ACC.NS          0.021785    0.029422  
           AIAENG.NS       0.002074   -0.002435  
...                             ...         ...  
2024-01-31 VMART.NS       -0.002896   -0.022373  
           VOLTAS.NS       0.032646    0.024002  
           WHIRLPOOL.NS    0.003024   -0.001913  
           ZFCVINDIA.NS    0.050616    0.046246  
           ZYDUSWELL.NS    0.006674    0.013290  

[3460 rows x 14 columns]

### Downloading Fama-French Factor